In [1]:
from copy import deepcopy
from time import perf_counter  # timing purposes only

import numpy as np
import pandas as pd
import os

from wombat import Simulation
from wombat.core.library import load_yaml

# Seed the random variable for consistently randomized results
# np.random.seed(0)

# Improve the legibility of DataFrames
pd.set_option("display.float_format", '{:,.2f}'.format)
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)

In [2]:
#path = "C:/Users/acooperm/Models/wombat06/WOMBAT/library/natLCOE/project/config"
#configs = os.listdir(path)
#configs = [c for c in configs if c.endswith(".yaml")]
#configs = [c.rstrip(".yaml") for c in configs]
#configs.remove("fixed_costs")
#print(configs)

configs = ["soATL20_fixed","soATL40_fixed","soATL60_fixed","soATL80_fixed"]

columns = deepcopy(configs) # Create a unique copy of the config names for column naming
results = {
    "availability - time based": [],
    "availability - production based": [],
    "capacity factor - net": [],
    "capacity factor - gross": [],
    "annual power production": [],
    "annual direct O&M cost": [],
    "annual vessel cost": [],
    "ctv cost": [],
    "sov cost": [],
    "hlv cost": [],
    "aht cost": [],
    "tug cost": [],
    "cable cost": [],
    "annual repair cost": [],
    "annual technician cost": [],
}

In [3]:
timing_df = pd.DataFrame([], columns=["Load Time (min)", "Run Time (min)"], index=configs)
timing_df.index.name = "Scenario"

for config in configs:
    # Seed the random variable for consistently randomized results
    np.random.seed(106)

    # Load the simulation
    start = perf_counter()
    sim = Simulation(library_path="natLCOE", config=f"{config}.yaml")
    end = perf_counter()
    timing_df.loc[config, "Load Time (min)"] = (end - start) / 60

    # Run the simulation
    start = perf_counter()
    sim.run()
    end = perf_counter()
    timing_df.loc[config, "Run Time (min)"] = (end - start) / 60

    # Gather the results of interest
    #years = sim.metrics.events.year.unique().shape[0]
    years = 6
    mil = 1000000

    # Gather the high-level results for the simulation
    availability_time = sim.metrics.time_based_availability(frequency="project", by="windfarm")
    availability_production = sim.metrics.production_based_availability(frequency="project", by="windfarm")
    cf_net = sim.metrics.capacity_factor(which="net", frequency="project", by="windfarm")
    cf_gross = sim.metrics.capacity_factor(which="gross", frequency="project", by="windfarm")
    power_production = sim.metrics.power_production(frequency="project", by="windfarm")
    parts = sim.metrics.events[["materials_cost"]].sum().sum()
    ops = sim.metrics.project_fixed_costs(frequency="project", resolution="low").operations[0]
    techs = sim.metrics.project_fixed_costs(frequency="project", resolution="medium").labor[0]
    total = sim.metrics.events[["total_cost"]].sum().sum()
    cap = 1000 * sim.metrics.project_capacity

    # Gather the equipment costs and separate the results by equipment type
    equipment = sim.metrics.equipment_costs(frequency="project", by_equipment=True)
    equipment_sum = equipment.sum().sum()
    sov = equipment[[el for el in equipment.columns if "SOV" in el]].sum().sum()
    aht = equipment[[el for el in equipment.columns if "Anchor" in el]].sum().sum()
    hlv = equipment[[el for el in equipment.columns if "Jack-up Vessel" in el]].sum().sum()
    tug = equipment[[el for el in equipment.columns if "Tugboat" in el]].sum().sum()
    ctv = equipment[[el for el in equipment.columns if "Crew Transfer Vessel" in el]].sum().sum()
    cable = equipment[[el for el in equipment.columns if "Cable Lay Vessel" in el]].sum().sum()
    
    # Log the results of interest
    results["availability - time based"].append(availability_time.values[0][0])
    results["availability - production based"].append(availability_production.values[0][0])
    results["capacity factor - net"].append(cf_net.values[0][0])
    results["capacity factor - gross"].append(cf_gross.values[0][0])
    results["annual power production"].append(power_production.values[0][0] / years)
    results["annual direct O&M cost"].append((total + ops) / cap / years)
    results["annual vessel cost"].append(equipment_sum / cap / years)
    results["sov cost"].append(sov / cap / years)
    results["aht cost"].append(aht / cap / years)
    results["hlv cost"].append(hlv / cap / years)
    results["ctv cost"].append(ctv / cap / years)
    results["tug cost"].append(tug / cap / years)
    results["cable cost"].append(cable / cap / years)
    results["annual repair cost"].append(parts / cap / years)
    results["annual technician cost"].append(techs / cap / years)
    
    # Clear the logs
    #sim.env.cleanup_log_files()

timing_df

,Load Time (min),Run Time (min)
Scenario,,
soATL20_fixed,0.16,1.57
soATL40_fixed,0.22,1.87
soATL60_fixed,0.31,2.18
soATL80_fixed,0.23,2.03


In [4]:
results_df = pd.DataFrame(results.values(), columns=columns, index=results.keys()).fillna(0)
results_df

,soATL20_fixed,soATL40_fixed,soATL60_fixed,soATL80_fixed
availability - time based,0.96,0.96,0.96,0.95
availability - production based,0.97,0.96,0.96,0.96
capacity factor - net,0.37,0.37,0.37,0.37
capacity factor - gross,0.38,0.38,0.38,0.38
annual power production,"3,235.05","3,229.78","3,224.00","3,217.28"
annual direct O&M cost,65.78,65.37,65.39,65.91
annual vessel cost,19.57,19.26,19.14,19.39
ctv cost,5.45,5.45,5.45,5.45
sov cost,0.00,0.00,0.00,0.00
hlv cost,9.21,9.25,8.95,9.18


In [5]:
results_df.to_csv("soATL_rs106_2.csv")